<a href="https://colab.research.google.com/github/gverafei/artificial-networks-technologies/blob/main/tarea7/model/tarea7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 7: Aplicación de CNN en Visión Artificial**

**Instrucciones:**

Aplique una CNN preentrenada, tales como VGGNet, ResNet, MobileNet, YOLO o cualquier otra que haya investigado para resolver un caso práctico de visión por computadora, utilizando técnicas de carga de modelos, extracción de características y fine-tuning.

El problema a resolver es libre.

Para este caso, se utilizara VGG-Net16 y ResNet50 preentrenadas para clasificar perros y gatos en tiempo real.

**Entregables**

+ Documento del trabajo (Springer)
+ Código de la implementación (.zip o link al repositorio en Github)

**La idea general:**

1. Cargar y preparar el dataset (train/val/test, prefetch, etc.).
2. Definir un bloque común de preprocesamiento / data augmentation.
3. Entrenar 4 modelos:
    + Modelo 1: MLP sin CNN (solo capas densas).
    + Modelo 2: CNN “desde cero”.
    + Modelo 3: VGG16 como extractor de características (congelada).
    + Modelo 4: VGG16 con fine-tuning.
    + Modelo 5: ResNet50 como extractor de características (congelada).
    + Modelo 6: ResNet50 con fine-tuning.
4. Registrar todo en TensorBoard.
5. Exportar el mejor modelo para usarlo en un sitio web.


## A. Configura el ambiente de pandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 3)
pd.set_option('display.width', 800)

print("Versiones -> pandas:", pd.__version__)

Versiones -> pandas: 2.2.2



## 1. Cargamos el dataset a utilizar


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [6]:
# Imports y configuración básica
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os
import datetime

print(tf.__version__)

2.19.0


In [7]:
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [8]:
!pip show keras

Name: keras
Version: 3.10.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: keras-hub, tensorflow


In [9]:
# No pude trabajar con el dataseet cats_vs_dogs de tensorflow datasets porque no cargaba en memoria local
# y en colab por reestricciones de memoria RAM.
# Así que descargue el dateset directamente desde Micrososft: https://www.microsoft.com/en-us/download/details.aspx?id=54765
# Y tomé las primeras 0-100 imagenes de cada clase y las puse en mi drive

# Así que usé una porción del dataset más pequeño localmente
TAMANO_IMG = 224      # VGG16 usa 224x224
BATCH_SIZE = 32
DATA_DIR = "/content/drive/MyDrive/cats_vs_dogs"
# DATA_DIR = "./cats_vs_dogs"   # Local
SEED = 42
EPOCHS = 5

Paso 1: Cargar dataset y dividir en train / val / test

1.1. Train y validation

In [10]:
# Carga los datos
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='training',
    seed=SEED,
    image_size=(TAMANO_IMG, TAMANO_IMG),
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='validation',
    seed=SEED,
    image_size=(TAMANO_IMG, TAMANO_IMG),
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Obtengo datasets cuyos elementos son: (batch_images, batch_labels)
# Cada batch_images tiene forma: (batch_size, 224, 224, 3)

Found 202 files belonging to 2 classes.
Using 162 files for training.
Found 202 files belonging to 2 classes.
Using 40 files for validation.


### 1.2. Separar un pequeño test a partir de val

Para no complicarnos, dividimos el dataset de validación en dos: mitad para validación real y mitad para test.

In [11]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)

print("Batches train:", tf.data.experimental.cardinality(train_ds).numpy())
print("Batches val:  ", tf.data.experimental.cardinality(val_ds).numpy())
print("Batches test: ", tf.data.experimental.cardinality(test_ds).numpy())


Batches train: 6
Batches val:   1
Batches test:  1


### 1.3. Optimizar el pipeline (cache / prefetch)

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.prefetch(buffer_size=AUTOTUNE)


## Paso 2: Data augmentation y preprocesamiento

Tendremos:

+ Un bloque de augmentación (rotación, zoom, flip).
+ Un Rescaling(1./255) para los modelos que no son VGG16.
+ Para VGG16 usaremos su propia función preprocess_input.

In [13]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation"
)

# Para modelos "desde cero" y CNN simple
rescale_layer = layers.Rescaling(1./255, name="rescale")

## Paso 3: Configurar TensorBoard y callbacks comunes

Vamos a crear una función que genera callbacks para cada experimento.

In [14]:
def get_callbacks(run_name):
    log_dir = os.path.join("logs", run_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    checkpoint_path = os.path.join("checkpoints", run_name + "_best.keras")
    ckpt_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    )

    early_stop = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    return [tb_callback, ckpt_callback, early_stop]

## Paso 4: Modelo 1 – Red sin CNN (MLP)

Aquí tomamos la imagen, la reescalamos y la aplanamos para alimentar una MLP.

Nota: No es la opción “ideal” para imágenes, pero sirve como baseline “sin CNN”.

In [ ]:
def build_model_mlp(input_shape=(TAMANO_IMG, TAMANO_IMG, 3)):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = rescale_layer(x)          # Escalar a [0,1]
    x = layers.Flatten()(x)       # Aplanar

    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="mlp_sin_cnn")
    return model

model_mlp = build_model_mlp()
model_mlp.summary()

model_mlp.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_mlp = get_callbacks("modelo_1_mlp")

history_mlp = model_mlp.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_mlp
)


Model: "mlp_sin_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescale (Rescaling)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    38,535,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,568,449 (147.13 MB)

 Trainable params: 38,568,449 (147.13 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5422 - auc: 0.5516 - loss: 2.7294 - precision: 0.5354 - recall: 0.4200
Epoch 1: val_accuracy improved from -inf to 0.62500, saving model to checkpoints/modelo_1_mlp_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 24s 3s/step - accuracy: 0.5388 - auc: 0.5448 - loss: 2.8607 - precision: 0.5314 - recall: 0.4251 - val_accuracy: 0.6250 - val_auc: 0.4375 - val_loss: 0.7242 - val_precision: 0.5714 - val_recall: 1.0000
Epoch 2/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.4238 - auc: 0.3889 - loss: 5.5907 - precision: 0.3903 - recall: 0.3660
Epoch 2: val_accuracy did not improve from 0.62500
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4420 - auc: 0.4104 - loss: 5.3933 - precision: 0.4137 - recall: 0.3844 - val_accuracy: 0.6250 - val_auc: 0.5333 - val_loss: 1.3331 - val_precision: 0.6250 - val_recall: 1.0000
Epoch 3/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.4426 - auc: 0.4698 - loss: 3.6897 - precision: 0.4535 - reca

## Paso 5: Modelo 2 – CNN “desde cero”

Ahora sí usamos capas convolucionales clásicas.

In [ ]:
def build_model_cnn(input_shape=(TAMANO_IMG, TAMANO_IMG, 3)):
    inputs = keras.Input(shape=input_shape)

    x = data_augmentation(inputs)
    x = rescale_layer(x)

    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="cnn_basica")
    return model

model_cnn = build_model_cnn()
model_cnn.summary()

model_cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_cnn = get_callbacks("modelo_2_cnn_basica")

history_cnn = model_cnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_cnn
)


Model: "cnn_basica"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescale (Rescaling)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,889 (429.25 KB)

 Trainable params: 109,889 (429.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.4803 - auc: 0.4379 - loss: 0.6955 - precision: 0.3803 - recall: 0.2758
Epoch 1: val_accuracy improved from -inf to 0.37500, saving model to checkpoints/modelo_2_cnn_basica_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 253ms/step - accuracy: 0.4769 - auc: 0.4374 - loss: 0.6957 - precision: 0.3831 - recall: 0.2690 - val_accuracy: 0.3750 - val_auc: 0.5667 - val_loss: 0.7007 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.5621 - auc: 0.5302 - loss: 0.6896 - precision: 0.4368 - recall: 0.2273
Epoch 2: val_accuracy improved from 0.37500 to 0.50000, saving model to checkpoints/modelo_2_cnn_basica_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.5567 - auc: 0.5402 - loss: 0.6900 - precision: 0.4742 - recall: 0.2383 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6934 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
5/6 ━━━━━━━━━━━━━━━━━━

## Paso 6: Modelo 3 – VGG16 como extractor de características (sin fine-tuning)

Para VGG16 usamos igualmente Rescaling(1./255) porque no podemos usar Lamnda en tensorflowjs

In [15]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

def build_model_vgg16_frozen(input_shape=(TAMANO_IMG, TAMANO_IMG, 3)):
    inputs = keras.Input(shape=input_shape)

    # Preprocesamiento sencillo (mejor soportado en TF.js)
    # x = layers.Rescaling(1./255, name="rescale")(inputs)
    x = data_augmentation(inputs)
    x = rescale_layer(x)          # Escalar a [0,1]

    #x = data_augmentation(inputs)
    # Convierte de [0,255] a lo que VGG16 espera
    #x = layers.Lambda(preprocess_input, name="vgg16_preprocess")(x)

    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    # Muy importante: congelar pesos del modelo base al inicio
    base_model.trainable = False   # Lo usamos sólo como extractor de características

    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)  # Global Average Pooling para pasar de feature maps a vector
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="vgg16_frozen")
    return model

model_vgg_frozen = build_model_vgg16_frozen()
model_vgg_frozen.summary()

model_vgg_frozen.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy', # Clasificación binaria
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_vgg_frozen = get_callbacks("modelo_3_vgg16_frozen")

history_vgg_frozen = model_vgg_frozen.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_vgg_frozen
)


58889256/58889256 [==============================] - 4s 0us/step
Model: "vgg16_frozen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescale (Rescaling)         (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               

## Paso 7: Modelo 4 – VGG16 con fine-tuning

Partimos del modelo anterior, pero ahora “descongelamos” las últimas capas de VGG16 (por ejemplo, el último bloque convolucional). Esto refina los pesos para nuestro problema específico.

In [ ]:
def build_model_vgg16_finetune(input_shape=(TAMANO_IMG, TAMANO_IMG, 3), fine_tune_at=15):
    inputs = keras.Input(shape=input_shape)

    # x = data_augmentation(inputs)
    # Preprocesamiento sencillo (mejor soportado en TF.js)
    # x = layers.Rescaling(1./255, name="rescale")(inputs)

    # Convierte de [0,255] a lo que VGG16 espera
    # x = layers.Lambda(preprocess_input, name="vgg16_preprocess")(x)

    # Preprocesamiento sencillo (mejor soportado en TF.js)
    x = data_augmentation(inputs)
    x = rescale_layer(x)          # Escalar a [0,1]

    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    # Primero descongelamos
    base_model.trainable = True

    # Congelamos todas las capas hasta `fine_tune_at`
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    print("Total capas VGG16:", len(base_model.layers))
    print("Fine-tuning a partir de la capa:", fine_tune_at, "-", base_model.layers[fine_tune_at].name)

    x = base_model(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="vgg16_finetune")
    return model

model_vgg_ft = build_model_vgg16_finetune()
model_vgg_ft.summary()

# Para fine-tuning usamos un learning rate más pequeño
model_vgg_ft.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_vgg_ft = get_callbacks("modelo_4_vgg16_finetune")

history_vgg_ft = model_vgg_ft.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_vgg_ft
)


Total capas VGG16: 19
Fine-tuning a partir de la capa: 15 - block5_conv1


Model: "vgg16_finetune"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescale (Rescaling)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16_preprocess (Lambda)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,846,273 (56.63 MB)

 Trainable params: 7,211,009 (27.51 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.6240 - auc: 0.5739 - loss: 0.7616 - precision: 0.6012 - recall: 0.3321 
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to checkpoints/modelo_4_vgg16_finetune_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 92s 15s/step - accuracy: 0.6168 - auc: 0.5709 - loss: 0.7722 - precision: 0.6046 - recall: 0.3299 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.7295 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.5278 - auc: 0.5304 - loss: 0.7964 - precision: 0.4757 - recall: 0.3213 
Epoch 2: val_accuracy did not improve from 0.50000
6/6 ━━━━━━━━━━━━━━━━━━━━ 87s 14s/step - accuracy: 0.5256 - auc: 0.5298 - loss: 0.8011 - precision: 0.4791 - recall: 0.3206 - val_accuracy: 0.3750 - val_auc: 0.6667 - val_loss: 0.7634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.5148 - auc: 0.5409 - loss: 0.7774 

## Paso 8: ResNet50 como extractor de características (sin fine-tuning)

Usamos ResNet50 preentrenada en ImageNet, congelada. Solo entrenamos la cabecera densa para tu problema de dos clases (perro/gato).

Qué estamos haciendo aquí:

+ Reutilizamos data_augmentation para generalizar mejor.
+ Aplicamos resnet_preprocess_input (normaliza como espera ResNet).
+ base_model.trainable = False → solo entrenas la parte densa final.
+ Guardamos logs y checkpoints con get_callbacks("modelo_5_resnet50_frozen") para ver todo en TensorBoard.

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input

def build_model_resnet_frozen(input_shape=(TAMANO_IMG, TAMANO_IMG, 3)):
    inputs = keras.Input(shape=input_shape)

    # Aumento de datos
    # x = data_augmentation(inputs)
    # Preprocesamiento específico de ResNet
    # x = layers.Lambda(resnet_preprocess_input, name="resnet_preprocess")(x)

    # Preprocesamiento sencillo (mejor soportado en TF.js)
    # x = layers.Rescaling(1./255, name="rescale")(inputs)
    x = data_augmentation(inputs)
    x = rescale_layer(x)          # Escalar a [0,1]

    # Modelo base ResNet50
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    base_model.trainable = False   # Congelado: no se actualizan sus pesos

    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="resnet50_frozen")
    return model

model_resnet_frozen = build_model_resnet_frozen()
model_resnet_frozen.summary()

model_resnet_frozen.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_resnet_frozen = get_callbacks("modelo_5_resnet50_frozen")

history_resnet_frozen = model_resnet_frozen.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_resnet_frozen
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Model: "resnet50_frozen"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet_preprocess (Lambda)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6266 - loss: 0.6910
Epoch 1: val_accuracy improved from -inf to 0.75000, saving model to checkpoints/modelo_5_resnet50_frozen_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.6253 - loss: 0.6905 - val_accuracy: 0.7500 - val_loss: 0.4430
Epoch 2/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.7349 - loss: 0.5628
Epoch 2: val_accuracy improved from 0.75000 to 1.00000, saving model to checkpoints/modelo_5_resnet50_frozen_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.7489 - loss: 0.5428 - val_accuracy: 1.0000 - val_loss: 0.1987
Epoch 3/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - accuracy: 0.8899 - loss: 0.2883
Epoch 3: val_accuracy did not improve from 1.00000
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.8931 - loss: 0.2880 - val_accuracy: 1.0000 - val_loss: 0.0900
Epoch 4/5
5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.9108 - loss: 0.3035
Epoch 4: val_accuracy did not improve from

## Paso 9: ResNet50 con fine-tuning

Ahora sí hacemos ajuste fino: descongelamos la parte final de ResNet para refinarla a perros vs gatos. Usamos un learning rate más pequeño.

Qué estamos haciendo aquí:

+ base_model.trainable = True → ahora sí permitimos ajustar pesos.
+ Calculamos fine_tune_at_layer_idx (por defecto toma las ~50 últimas capas).
+ Congelamos las capas antes de ese índice y dejamos entrenables las últimas.
+ Learning rate pequeño (1e-5) para no “destrozar” lo aprendido en ImageNet.

In [ ]:
def build_model_resnet_finetune(input_shape=(TAMANO_IMG, TAMANO_IMG, 3),
                                fine_tune_at_layer_idx=None):
    inputs = keras.Input(shape=input_shape)

    # x = data_augmentation(inputs)
    # x = layers.Lambda(resnet_preprocess_input, name="resnet_preprocess")(x)

    # Preprocesamiento sencillo (mejor soportado en TF.js)
    # x = layers.Rescaling(1./255, name="rescale")(inputs)
    x = data_augmentation(inputs)
    x = rescale_layer(x)          # Escalar a [0,1]

    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )

    # Hacemos el modelo base entrenable
    base_model.trainable = True

    # Si no se especifica índice, descongelamos más o menos el último 30% de las capas
    if fine_tune_at_layer_idx is None:
        fine_tune_at_layer_idx = len(base_model.layers) - 50  # puedes ajustar este número

    print("Total capas ResNet50:", len(base_model.layers))
    print("Fine-tuning a partir de la capa:", fine_tune_at_layer_idx, "-", base_model.layers[fine_tune_at_layer_idx].name)

    # Congelamos las primeras capas, dejamos libres las últimas
    for layer in base_model.layers[:fine_tune_at_layer_idx]:
        layer.trainable = False

    x = base_model(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs, outputs, name="resnet50_finetune")
    return model

model_resnet_ft = build_model_resnet_finetune()
model_resnet_ft.summary()

# Learning rate más pequeño para fine-tuning
model_resnet_ft.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc")
    ]
)

callbacks_resnet_ft = get_callbacks("modelo_6_resnet50_finetune")

history_resnet_ft = model_resnet_ft.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks_resnet_ft
)


Total capas ResNet50: 175
Fine-tuning a partir de la capa: 125 - conv4_block5_1_relu


Model: "resnet50_finetune"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet_preprocess (Lambda)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 17,475,329 (66.66 MB)

 Non-trainable params: 6,637,184 (25.32 MB)

Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.4937 - auc: 0.4560 - loss: 0.9191 - precision: 0.4701 - recall: 0.3842
Epoch 1: val_accuracy improved from -inf to 0.62500, saving model to checkpoints/modelo_6_resnet50_finetune_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.4964 - auc: 0.4620 - loss: 0.9105 - precision: 0.4743 - recall: 0.3890 - val_accuracy: 0.6250 - val_auc: 0.6000 - val_loss: 0.6622 - val_precision: 0.6667 - val_recall: 0.8000
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.6367 - auc: 0.6566 - loss: 0.6798 - precision: 0.6135 - recall: 0.5557
Epoch 2: val_accuracy improved from 0.62500 to 0.75000, saving model to checkpoints/modelo_6_resnet50_finetune_best.keras
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6304 - auc: 0.6536 - loss: 0.6814 - precision: 0.6107 - recall: 0.5505 - val_accuracy: 0.7500 - val_auc: 0.5333 - val_loss: 0.6678 - val_precision: 0.7143 - val_recall: 1.0000
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s

## Paso 8: Visualizar resultados en TensorBoard

En una celda nueva de Colab:

In [ ]:
#Cargar la extension de tensorboard de colab
%load_ext tensorboard

In [ ]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs"
%tensorboard --logdir logs --port=6005

Esto nos permite comparar las 4 corridas (loss, accuracy, etc.) y sacar gráficas para el reporte escrito.

## Paso 9: Evaluar en el set de prueba

Ahora podemos escoger cada modelo y evaluarlo:

In [ ]:
print("MLP en test:")
model_mlp.evaluate(test_ds)

print("CNN básica en test:")
model_cnn.evaluate(test_ds)

print("VGG16 frozen en test:")
model_vgg_frozen.evaluate(test_ds)

print("VGG16 fine-tune en test:")
model_vgg_ft.evaluate(test_ds)

print("ResNet50 frozen en test:")
model_resnet_frozen.evaluate(test_ds)

print("ResNet50 fine-tune en test:")
model_resnet_ft.evaluate(test_ds)


In [16]:
print("VGG16 fine-tune en test:")
model_vgg_frozen.evaluate(test_ds)

VGG16 fine-tune en test:
1/1 [==============================] - 0s 168ms/step - loss: 0.6607 - accuracy: 0.7500 - precision: 0.9167 - recall: 0.6111 - auc: 0.8214


[0.6607002019882202,
 0.75,
 0.9166666865348816,
 0.6111111044883728,
 0.8214285373687744]

## Paso 10: Exportar el mejor modelo para usarlo en web

Supongamos que el mejor fue la VGG16 con fine-tuning. Podemos guardar el modelo completo:

In [17]:
!pip install tensorflowjs --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.38.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
xarray 2025.11.0 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.4 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [18]:
import tensorflowjs as tfjs
import keras

tfjs_target_dir = 'model_vgg_frozen'
tfjs.converters.save_keras_model(model_vgg_frozen, tfjs_target_dir)

/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Finalmente se descarga el modelo para su uso en una aplicación web

In [19]:
!zip -r model_vgg_frozen.zip model_vgg_frozen

  adding: model_vgg_frozen/ (stored 0%)
  adding: model_vgg_frozen/group1-shard4of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard11of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard8of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard15of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard13of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard2of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard12of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard3of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard1of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard5of15.bin (deflated 7%)
  adding: model_vgg_frozen/model.json (deflated 90%)
  adding: model_vgg_frozen/group1-shard14of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard10of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard6of15.bin (deflated 7%)
  adding: model_vgg_frozen/group1-shard9of15.bin (deflated 7%)
  adding: model_vgg